# Backend.AI 체험하기
### pytorch version (Custom)
DATASET : FashionMNIST
MODEL : LeNET

### Improve performance
1. Change Hyper Parameters
2. Change Model DNN -> CNN (use GPU in Backend.AI cloud)

In [42]:
import numpy as np
import torch
from torchvision import datasets, transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm

config = {
    'batch_size' : 64,
    'lr':0.001,
    'n_classes' : 10,
    'epochs':20,
    'mean':0.5, 
    'std':0.5,
    'device': 'cuda:0' if torch.cuda.is_available() else 'cpu'
}

In [43]:

'''
# 간단한 transform 정의
'''
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((config['mean']), (config['std']))])


'''
# dataset & dataLoader
'''
trainset = datasets.FashionMNIST('.', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=config['batch_size'], shuffle=True)


testset = datasets.FashionMNIST('.', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=config['batch_size'], shuffle=False)

In [44]:
'''
# class balance 조사

balance good!!

'''
class_dict = {}
for _class in trainset.targets:
    _class = _class.item()
    if _class not in class_dict:
        class_dict[_class]=0
    class_dict[_class]+=1
print(class_dict)
config['n_classes'] = len(class_dict.keys())
config['n_classes']

{9: 6000, 0: 6000, 3: 6000, 2: 6000, 7: 6000, 5: 6000, 1: 6000, 6: 6000, 4: 6000, 8: 6000}


10

In [45]:
'''
MODEL 정의
tensorflow 예제에서는 단순 DNN이였다면 
퍼포먼스 향상을 위해 CNN(LeNet)으로 change
'''
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, n_classes = 1):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1,6,kernel_size = 1) # 28 x 28 이므로 5->1로 변경
        self.conv2 = nn.Conv2d(6,16,kernel_size = 5)
        self.conv3 = nn.Conv2d(16,120,kernel_size = 5)
        self.fc1 = nn.Linear(120, 84)
        self.fc2 = nn.Linear(84, n_classes)
        self.pool = nn.MaxPool2d(kernel_size = 2, stride =2)
        
    def forward(self, x): # tanh -> relu
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = x.view(-1,120)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

class DNN(nn.Module):
    def __init__(self, n_classes = 1):
        super(DNN, self).__init__()
        self.fc1 = nn.Linear(28*28,128)
        self.fc2 = nn.Linear(128, 10)
    
    def forward(self, x):
        x = x.view(-1,28*28)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [46]:
'''
train, valdation function
'''
def train(model, dataloader, criterion, optimizer, device):
    running_loss = 0
    for images, labels in tqdm(dataloader, position=0, leave=True):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    running_loss /= len(dataloader)
    return running_loss

def validation(model, dataloader, criterion, device):
    running_loss = 0
    preds = []
    targets = []
    for images, labels in tqdm(dataloader, position=0, leave=True):
        with torch.no_grad():
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
        
        preds += torch.argmax(outputs,1).tolist()
        targets += labels.tolist()
        running_loss += loss.item()
    preds = np.array(preds)
    targets = np.array(targets)
    score = (preds == targets).sum() / len(preds)
    running_loss /= len(dataloader)
    return running_loss, score

In [47]:
'''
CNN 실험해보기
'''
device = config['device']
model = CNN(config['n_classes']).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = config['lr'])
epochs = config['epochs']


for epoch in range(1, epochs):
    t_loss = train(model, trainloader, criterion, optimizer, device)
    v_loss, score = validation(model, testloader, criterion, device)
    print('train_loss : {:.4f} \t test_loss : {:.4f} \t score : {:.3f}'.format(t_loss, v_loss, score))
print("Accuracy Score : {}".format(score))

  1%|          | 11/938 [00:00<00:08, 103.89it/s]

train_loss : 0.6861 	 test_loss : 0.5520 	 score : 0.792


  1%|          | 11/938 [00:00<00:09, 100.32it/s]

train_loss : 0.4511 	 test_loss : 0.4154 	 score : 0.849


  1%|          | 9/938 [00:00<00:11, 81.83it/s]

...

train_loss : 0.2063 	 test_loss : 0.3174 	 score : 0.894


  1%|          | 11/938 [00:00<00:08, 107.87it/s]

train_loss : 0.2018 	 test_loss : 0.3077 	 score : 0.895


100%|██████████| 157/157 [00:01<00:00, 118.78it/s]

train_loss : 0.1955 	 test_loss : 0.3122 	 score : 0.895
Accuracy Score : 0.895


In [48]:
'''
DNN 실험해보기
'''
device = config['device']
model = DNN(config['n_classes']).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = config['lr'])
epochs = config['epochs']


for epoch in range(1, epochs):
    t_loss = train(model, trainloader, criterion, optimizer, device)
    v_loss, score = validation(model, testloader, criterion, device)
    print('train_loss : {:.4f} \t test_loss : {:.4f} \t score : {:.3f}'.format(t_loss, v_loss, score))
print("Accuracy Score : {}".format(score))

  1%|          | 11/938 [00:00<00:08, 109.26it/s]

train_loss : 0.4994 	 test_loss : 0.4392 	 score : 0.842


  1%|▏         | 13/938 [00:00<00:07, 126.79it/s]

train_loss : 0.3768 	 test_loss : 0.3998 	 score : 0.855


  1%|▏         | 12/938 [00:00<00:08, 115.71it/s]

...

train_loss : 0.1917 	 test_loss : 0.3516 	 score : 0.888


  1%|▏         | 12/938 [00:00<00:07, 116.82it/s]

train_loss : 0.1862 	 test_loss : 0.3546 	 score : 0.884


100%|██████████| 157/157 [00:01<00:00, 121.97it/s]

train_loss : 0.1813 	 test_loss : 0.3676 	 score : 0.886
Accuracy Score : 0.886


# Result

에폭 늘려서 추가 학습을 진행시킬 경우 CNN이 DNN보다 약 0.01정도 높음.   
backend.ai pytorch-gpu 환경도 적응완료!